In [1]:
library(plyr)
library(tidyverse)
library(readxl)

── Attaching core tidyverse packages ─────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ───────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::arrange()   masks plyr::arrange()
✖ purrr::compact()   masks plyr::compact()
✖ dplyr::count()     masks plyr::count()
✖ dplyr::desc()      masks plyr::desc()
✖ dplyr::failwith()  masks plyr::failwith()
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::id()        masks plyr::id()
✖ dplyr::lag()       masks stats::lag()
✖ dplyr::mutate()    masks plyr::mutate()
✖ dplyr::rename()    masks plyr::rename()
✖ dplyr::summarise() masks plyr::summarise()
✖ dplyr::summarize() masks plyr::summarize()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become 

In [2]:
source("utils/Task2_helperFunctions.r")

In [3]:
getwd()

[1] "/Users/salkhairy/Desktop/projects/llm_evaluation_for_gene_set_interpretation"

In [4]:
mapToHigher = list("LLM" = "At least as good",
                   "Tied" = "At least as good",
                   "GO" = "At least as bad",
                   "Neither" = "At least as bad")

In [5]:
mapBlinded = list("Name B" = "LLM",
                  "Name A" = "GO",
                 "Neither" = "Neither",
                 "Tied" = "Tied")

### Read in SapBERT determination

In [6]:
sapbert_DF = read_delim(file = "data/omics_LLM_Enrichr_simVals_analyses_DF.tsv", delim = "\t")

Rows: 250 Columns: 28
── Column specification ───────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (11): Source, GeneSetID, GeneSetName, GeneList, LLM Name, LLM Analysis, ...
dbl (17): ...1, index, n_Genes, Score, Rank, P-value, Adjusted P-value, Orig...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


### Read in consensus analyses

In [7]:
human_consensus_DF = read_xlsx(path = "data/Omics - blinded study.xlsx", sheet =  "Merged")

In [8]:
colnames(human_consensus_DF)

[1] "Source"             "GeneSetID"          "GeneSetName"       
 [4] "n_Genes"            "GeneList"           "Name A"            
 [7] "Name B"             "Winner_human1"      "Winner_human2"     
[10] "Winner_human3"      "OverridingDecision"

### Merge and analyse consensus

In [9]:
merged_DF = inner_join(x = sapbert_DF, y = human_consensus_DF, by = c("Source", "GeneSetID", "GeneSetName", "GeneList"))

In [10]:
merged_DF = merged_DF %>% 
rowwise() %>%
mutate(humanConsensusBlinded = getHumanConsensus(Winner_human1,Winner_human2,  Winner_human3, OverridingDecision),
       humanConsensus = mapBlinded[as.character(`humanConsensusBlinded`)],
        humanConsensus_higherLevel = mapToHigher[as.character(`humanConsensus`)],
       winner_higherLevel = mapToHigher[as.character(`winner`)])  %>%
select(c("Source", "GeneSetID", "GeneSetName", "GeneList", "winner", "humanConsensus" , "OverridingDecision", 
        "humanConsensus_higherLevel", "winner_higherLevel"))



In [12]:
merged_DF$winner_higherLevel <- factor(merged_DF$winner_higherLevel, levels=c("At least as good", "At least as bad"))


In [13]:
merged_DF$humanConsensus_higherLevel <- factor(merged_DF$humanConsensus_higherLevel, levels=c("At least as good", "At least as bad"))


In [28]:
merged_DF[,c("humanConsensus_higherLevel", "winner_higherLevel")]

humanConsensus_higherLevel,winner_higherLevel
<fct>,<fct>
At least as bad,At least as bad
At least as good,At least as bad
At least as good,At least as good
At least as good,At least as good
At least as bad,At least as bad
At least as good,At least as bad
At least as good,At least as bad
At least as bad,At least as good
At least as bad,At least as bad


In [21]:
 higherLevel_contingencyTable  = table(merged_DF[,c("humanConsensus_higherLevel", "winner_higherLevel")])

In [22]:
higherLevel_contingencyTable

                          winner_higherLevel
humanConsensus_higherLevel At least as good At least as bad
          At least as good               22              14
          At least as bad                 5               9

In [25]:
(higherLevel_contingencyTable[1,1] + higherLevel_contingencyTable[2,2]) /sum(higherLevel_contingencyTable)

[1] 0.62

In [27]:
(higherLevel_contingencyTable[1,1] + higherLevel_contingencyTable[2,2]) 

[1] 31